In [1]:
# import libraries
import nltk

In [5]:
import re
import pandas as pd
from transformers import BertTokenizer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from transformers import BertTokenizer
from spellchecker import SpellChecker
from nltk.tokenize import word_tokenize
spell = SpellChecker()
import swifter
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud, STOPWORDS
import plotly.express as px
import numpy as np
import emoji
from transformers import AutoTokenizer

In [6]:
# file_path = "/content/interDataset.csv"
file_path = "./finalOne.csv"

# Load the dataset
dataframe = pd.read_csv(file_path)

# Display basic information
dataframe.head()

,comment_likes,short_code,productName,productLikes,productUrl,comment_body,title,comment_year,comment_month,category,source,tokens_ids,tokens_text
0,59,4000270696,"LG 77"" Class - OLED B4 Series - 4K UHD OLED TV...",98,https://www.costco.com/lg-77-class---oled-b4-s...,the picture is beautiful . four stars because ...,"Great picture, slow Ethernet, good wifi",2024,12,Product,Costco,"[0, 627, 2170, 16, 2721, 479, 237, 2690, 142, ...","['<s>', 'the', 'Ġpicture', 'Ġis', 'Ġbeautiful'..."
1,3,4000270696,"LG 77"" Class - OLED B4 Series - 4K UHD OLED TV...",98,https://www.costco.com/lg-77-class---oled-b4-s...,"i recently upgraded to the la old 77 b4 , and ...",Stunning Picture Quality and Performance,2025,1,Product,Costco,"[0, 118, 682, 7084, 7, 5, 897, 793, 6791, 741,...","['<s>', 'i', 'Ġrecently', 'Ġupgraded', 'Ġto', ..."
2,12,4000270696,"LG 77"" Class - OLED B4 Series - 4K UHD OLED TV...",98,https://www.costco.com/lg-77-class---oled-b4-s...,"great picture , but doesn't play well in with ...",Great picture OS app integration so so,2024,12,Product,Costco,"[0, 12338, 2170, 2156, 53, 630, 75, 310, 157, ...","['<s>', 'great', 'Ġpicture', 'Ġ,', 'Ġbut', 'Ġd..."
3,8,4000270696,"LG 77"" Class - OLED B4 Series - 4K UHD OLED TV...",98,https://www.costco.com/lg-77-class---oled-b4-s...,"costco was great about honoring the warranty ,...",Love it when it's working right,2025,1,Product,Costco,"[0, 10111, 876, 21, 372, 59, 17198, 5, 21784, ...","['<s>', 'cost', 'co', 'Ġwas', 'Ġgreat', 'Ġabou..."
4,1,4000270696,"LG 77"" Class - OLED B4 Series - 4K UHD OLED TV...",98,https://www.costco.com/lg-77-class---oled-b4-s...,sweet . great picture and relatively easy to i...,Sweet.,2025,1,Product,Costco,"[0, 32588, 479, 372, 2170, 8, 3487, 1365, 7, 8...","['<s>', 'sweet', 'Ġ.', 'Ġgreat', 'Ġpicture', '..."


In [ ]:
from transformers import pipeline

# Grouping the dataframe by product and year, combining all comments into one long string
grouped = dataframe.groupby(['productName', 'comment_year'])['comment_body'] \
    .apply(lambda x: ' '.join(x.dropna().astype(str))) \
    .reset_index()

grouped.head(15)

,productName,comment_year,comment_body
0,10' X 20' Steel Frame Canopy,2021,"the canopy covers our boat , but the condensat..."
1,10' X 20' Steel Frame Canopy,2022,super happy with this ! we had rain and strong...
2,10' X 20' Steel Frame Canopy,2023,this is a heavy duty storage tent the poles ar...
3,10' X 20' Steel Frame Canopy,2024,these are the best ones on the market for thei...
4,10' X 20' Steel Frame Canopy,2025,"i put the canopy up by myself , which says lot..."
5,"93"" Plush Bear, Blonde",2019,priceless section reaction from recipient boug...
6,"93"" Plush Bear, Blonde",2020,this giant bear is the bomb ! my grandson is o...
7,"93"" Plush Bear, Blonde",2021,i purchased this for my soon to arrive grandso...
8,"93"" Plush Bear, Blonde",2022,my granddaughter received one of the smaller v...
9,"93"" Plush Bear, Blonde",2023,its huge ! the biggest bear and at a decent pr...


In [ ]:
# Filter for specific product and year
filtered_comments = dataframe[
    (dataframe['productName'] == "10' X 20' Steel Frame Canopy") & (dataframe['comment_year'] == 2021)
]

# Print each comment
for idx, comment in enumerate(filtered_comments['comment_body'].dropna(), start=1):
    print(f"{idx}. {comment}")

1. the canopy covers our boat , but the condensation makes it necessary to put an additional cover over the boat .
2. great canopy , took no time to assemble and does the job required .
3. i have had these before , and they are excellent product .
4. so far give these . easy install , sturdy and durable .
5. good price and quality .


In [ ]:
grouped.iloc[0]['comment_body']

'the canopy covers our boat , but the condensation makes it necessary to put an additional cover over the boat . great canopy , took no time to assemble and does the job required . i have had these before , and they are excellent product . so far give these . easy install , sturdy and durable . good price and quality .'

In [ ]:
from transformers import pipeline, AutoTokenizer

# Load summarization pipeline and tokenizer
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")
tokenizer = AutoTokenizer.from_pretrained("facebook/bart-large-cnn")

# Define summarization function with token-safe truncation
def summarize_text(text, max_tokens=1024):
    if not text or not isinstance(text, str) or not text.strip():
        return "No valid comment text to summarize."

    try:
        # Tokenize and truncate
        inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=max_tokens)
        truncated_text = tokenizer.decode(inputs['input_ids'][0], skip_special_tokens=True)

        # Estimate a sensible min length
        word_count = len(truncated_text.split())
        min_len = min(15, word_count // 2)

        # Summarize
        summary = summarizer(truncated_text, max_length=80, min_length=min_len, do_sample=False)

        if summary and isinstance(summary, list) and 'summary_text' in summary[0]:
            return summary[0]['summary_text']
        else:
            return "Summarization failed or returned no output."

    except Exception as e:
        return f"Summarization error: {str(e)}"

# Build summary dictionary
summary_dict = {}

for _, row in grouped.iterrows():
    product = row['productName']
    year = row['comment_year']
    combined_reviews = row['comment_body']

    if not isinstance(combined_reviews, str) or not combined_reviews.strip():
        summary = "No comments available."
    else:
        summary = summarize_text(combined_reviews)

    key = f"{product}_{year}"
    summary_dict[key] = summary

In [ ]:
# Print sample output
for i, (key, value) in enumerate(summary_dict.items()):
    if i >= 10:
        break
    print(f"{key}: {value}")

10' X 20' Steel Frame Canopy_2021: Great canopy , took no time to assemble and does the job required. easy install, sturdy and durable . good price and quality.
10' X 20' Steel Frame Canopy_2022: This is a made in china structure that does not really disappoint . it assembles well . comes with everything but will still blow away if you do not anchor it with other anchors and ratchet straps. We use it for a year or more . we installed it at our camper . bought to use as a storage unit . the price is very reasonable . came with everything we needed
10' X 20' Steel Frame Canopy_2023: i have been waiting a number of years for this to come back and happy they did . i have 4 of these . the only problem is getting replacement sides and front and rear doors. elastic ties might be ok for milder climates , but they do not hold well with a light 3 snow load. i will replace them with paranoid ties and that should remedy the issue. buy it now
10' X 20' Steel Frame Canopy_2024: i have bought 4 over 

In [ ]:
print(summary_dict["10' X 20' Steel Frame Canopy_2021"])

Great canopy , took no time to assemble and does the job required. easy install, sturdy and durable . good price and quality.


In [ ]:
# Define a list of error indicators
error_messages = [
    "No valid comment text to summarize.",
    "Summarization failed or returned no output.",
    "No comments available."
]

# Create a list to collect problematic entries
error_entries = [
    (key, value)
    for key, value in summary_dict.items()
    if (
        value in error_messages or
        value.startswith("Summarization error:")
    )
]

# Print the count
print(f"Number of summarization issues: {len(error_entries)}")

Number of summarization issues: 0


In [ ]:
print(f"Total entries in summary_dict: {len(summary_dict)}")

Total entries in summary_dict: 1743


In [ ]:
# Show full column content without truncation
#pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_colwidth', 50)

In [ ]:
import json

# Save to JSON
with open("summary_dict.json", "w") as f:
    json.dump(summary_dict, f)

print("summary_dict saved to summary_dict.json")

summary_dict saved to summary_dict.json


In [ ]:
# Load from JSON
with open("summary_dict.json", "r") as f:
    summary_dict = json.load(f)

# Print sample output
for i, (key, value) in enumerate(summary_dict.items()):
    if i >= 2:
        break
    print(f"{key}: {value}")

10' X 20' Steel Frame Canopy_2021: Great canopy , took no time to assemble and does the job required. easy install, sturdy and durable . good price and quality.
10' X 20' Steel Frame Canopy_2022: This is a made in china structure that does not really disappoint . it assembles well . comes with everything but will still blow away if you do not anchor it with other anchors and ratchet straps. We use it for a year or more . we installed it at our camper . bought to use as a storage unit . the price is very reasonable . came with everything we needed
